In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # or "3" for GPU 3


In [ ]:
# DOF_Feature_Extraction.ipynb (Notebook 2)

import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
from transformers import TFViTForImageClassification, ViTImageProcessor

print(f"\U0001F680 Starting DOF feature extraction at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

image_dir = "/home/shahid/000.Damage Severity/aman/"
model_name = "microsoft/swin-tiny-patch4-window7-224"
model_save_path = "./swin_transformer_dof"

df_train = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_train.tsv"))
df_val   = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_dev.tsv"))
df_test  = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_test.tsv"))
label_mapping = {"severe_damage": 0, "mild_damage": 1, "little_or_no_damage": 2}
for df in [df_train, df_val, df_test]:
    df["damage_label"] = df["label"].map(label_mapping)

processor = ViTImageProcessor.from_pretrained(model_name)
model = TFViTForImageClassification.from_pretrained(model_name, num_labels=3, ignore_mismatched_sizes=True)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

if not os.path.exists(model_save_path):
    model.save(model_save_path)

def load_image(image_path):
    full_path = os.path.join(image_dir, image_path)
    if not os.path.exists(full_path):
        return None, None
    img = cv2.imread(full_path)
    if img is None:
        return None, None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (224, 224))
    inputs = processor(images=img_resized, return_tensors="tf")
    return img_resized, inputs["pixel_values"]

# === DOF computation ===
def compute_dof(image, img_tensor):
    patch_size = 32
    stride = 16
    sensitivity_map = np.zeros((224, 224))
    base_pred = model(img_tensor).logits[0, 0].numpy()
    for i in range(0, 224 - patch_size + 1, stride):
        for j in range(0, 224 - patch_size + 1, stride):
            img_copy = img_tensor.numpy().copy()
            img_copy[0, i:i+patch_size, j:j+patch_size, :] = 0
            occluded_pred = model(img_copy).logits[0, 0].numpy()
            sensitivity_map[i:i+patch_size, j:j+patch_size] = np.abs(base_pred - occluded_pred)
    features = [np.mean(sensitivity_map), np.max(sensitivity_map), np.std(sensitivity_map)]
    return np.nan_to_num(features), sensitivity_map

def extract_dof_features(df, split_name):
    features = []
    for idx, row in df.iterrows():
        print(f"[{idx + 1}/{len(df)}] Processing: {row['image']}")
        img, img_tensor = load_image(row['image'])
        if img is None or img_tensor is None:
            continue
        dof_feat, _ = compute_dof(img, img_tensor)
        features.append(dof_feat)
        print(f"    ✅ Feature extracted | Mean: {dof_feat[0]:.4f}")
    features = np.array(features)
    np.save(f"{split_name}_dof_features.npy", features)
    print(f"Saved {split_name} DOF features: {features.shape} → {split_name}_dof_features.npy")
    return features

train_features = extract_dof_features(df_train, "train")
val_features   = extract_dof_features(df_val, "val")
test_features  = extract_dof_features(df_test, "test")

print("\n✅ DOF feature extraction complete!")

/home/shahid/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2025-07-02 17:02:53.508101: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-02 17:02:53.548782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-02 17:02:53.593793: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-02 17:02:53.610536: E external/local_xla/xla/stream_exe

🚀 Starting DOF feature extraction at 2025-07-02 17:03:14


You are using a model of type swin to instantiate a model of type vit. This is not supported for all configurations of models and can yield errors.
2025-07-02 17:03:15.657204: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-07-02 17:03:18.453091: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Input to reshape is a tensor with 768000 values, but the requested shape has 2304
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['swin.encoder.layers.1.downsample.norm.weight', 'swin.encoder.layers.1.blocks.0.output.dense.weight', 'swin.encoder.layers

[1/2468] Processing: data_image/hurricane_harvey/8_9_2017/905960092822003712_0.jpg
    ✅ Feature extracted | Mean: 0.0025
[2/2468] Processing: data_image/california_wildfires/11_10_2017/918008272363368448_0.jpg
    ✅ Feature extracted | Mean: 0.0029
[3/2468] Processing: data_image/hurricane_irma/17_9_2017/909396901254090752_0.jpg
    ✅ Feature extracted | Mean: 0.0015
[4/2468] Processing: data_image/hurricane_maria/24_9_2017/912097936200355841_0.jpg
    ✅ Feature extracted | Mean: 0.0037
[5/2468] Processing: data_image/hurricane_maria/23_10_2017/922610290281402368_1.jpg
    ✅ Feature extracted | Mean: 0.0004
[9/2468] Processing: data_image/hurricane_harvey/10_9_2017/906792024094986240_0.jpg
    ✅ Feature extracted | Mean: 0.0010
[10/2468] Processing: data_image/hurricane_irma/19_9_2017/910118839081472000_0.jpg
    ✅ Feature extracted | Mean: 0.0011
[11/2468] Processing: data_image/iraq_iran_earthquake/18_11_2017/931898198569996288_0.jpg
    ✅ Feature extracted | Mean: 0.0033
[12/2468] 

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from datetime import datetime
from transformers import TFViTForImageClassification, ViTImageProcessor
from concurrent.futures import ThreadPoolExecutor, as_completed

print(f"\U0001F680 Starting DOF feature extraction at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

image_dir = "/home/shahid/000.Damage Severity/aman/"
model_name = "microsoft/swin-tiny-patch4-window7-224"
model_save_path = "./swin_transformer_dof"

# Load data
df_train = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_train.tsv"))
df_val = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_dev.tsv"))
df_test = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_test.tsv"))
label_mapping = {"severe_damage": 0, "mild_damage": 1, "little_or_no_damage": 2}
for df in [df_train, df_val, df_test]:
    df["damage_label"] = df["label"].map(label_mapping)

# Load model and processor
processor = ViTImageProcessor.from_pretrained(model_name)
model = TFViTForImageClassification.from_pretrained(model_name, num_labels=3, ignore_mismatched_sizes=True)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

if not os.path.exists(model_save_path):
    model.save(model_save_path)

def load_image(image_path):
    full_path = os.path.join(image_dir, image_path)
    if not os.path.exists(full_path):
        return None, None
    img = cv2.imread(full_path)
    if img is None:
        return None, None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (224, 224))
    inputs = processor(images=img_resized, return_tensors="tf")
    return img_resized, inputs["pixel_values"]

def compute_dof(image, img_tensor):
    patch_size = 64  # Larger patch size
    stride = 32      # Larger stride
    sensitivity_map = np.zeros((224, 224))
    base_pred = model(img_tensor).logits[0, 0].numpy()

    patches = []
    patch_coords = []
    for i in range(0, 224 - patch_size + 1, stride):
        for j in range(0, 224 - patch_size + 1, stride):
            patches.append(img_tensor.numpy().copy())
            patch_coords.append((i, j))
    
    batch_patches = np.stack(patches, axis=0)
    for idx, (i, j) in enumerate(patch_coords):
        batch_patches[idx, :, i:i+patch_size, j:j+patch_size, :] = 0
    
    occluded_preds = model(batch_patches).logits[:, 0].numpy()
    
    for idx, (i, j) in enumerate(patch_coords):
        sensitivity_map[i:i+patch_size, j:j+patch_size] = np.abs(base_pred - occluded_preds[idx])
    
    features = [np.mean(sensitivity_map), np.max(sensitivity_map), np.std(sensitivity_map)]
    return np.nan_to_num(features), sensitivity_map

def process_single_image(row):
    print(f"Processing: {row['image']}")
    img, img_tensor = load_image(row['image'])
    if img is None or img_tensor is None:
        return None
    dof_feat, _ = compute_dof(img, img_tensor)
    print(f"    ✅ Feature extracted | Mean: {dof_feat[0]:.4f}")
    return dof_feat

def extract_dof_features(df, split_name, max_workers=4):
    output_file = f"{split_name}_dof_features.npy"
    if os.path.exists(output_file):
        print(f"Loading cached {split_name} DOF features from {output_file}")
        return np.load(output_file)
    
    features = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_row = {executor.submit(process_single_image, row): row for _, row in df.iterrows()}
        for future in as_completed(future_to_row):
            result = future.result()
            if result is not None:
                features.append(result)
    
    features = np.array(features)
    np.save(output_file, features)
    print(f"Saved {split_name} DOF features: {features.shape} → {output_file}")
    return features

train_features = extract_dof_features(df_train, "train")
val_features = extract_dof_features(df_val, "val")
test_features = extract_dof_features(df_test, "test")

print("\n✅ DOF feature extraction complete!")

/home/shahid/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2025-07-03 09:58:00.431471: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 09:58:00.495360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 09:58:00.530832: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 09:58:00.540541: E external/local_xla/xla/stream_exe

🚀 Starting DOF feature extraction at 2025-07-03 09:58:20


You are using a model of type swin to instantiate a model of type vit. This is not supported for all configurations of models and can yield errors.
2025-07-03 09:58:21.150550: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-07-03 09:58:23.417049: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Input to reshape is a tensor with 768000 values, but the requested shape has 2304
2025-07-03 09:58:23.426617: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Input to reshape is a tensor with 1000 values, but the requested shape has 3
Some weights of t

Processing: data_image/hurricane_harvey/8_9_2017/905960092822003712_0.jpg
Processing: data_image/california_wildfires/11_10_2017/918008272363368448_0.jpg
Processing: data_image/hurricane_irma/17_9_2017/909396901254090752_0.jpg
Processing: data_image/hurricane_maria/24_9_2017/912097936200355841_0.jpg
Processing: data_image/hurricane_maria/23_10_2017/922610290281402368_1.jpg
Processing: data_image/california_wildfires/15_10_2017/919585500914208768_0.jpg
Processing: data_image/hurricane_maria/1_10_2017/914372204787355648_0.jpg
Processing: data_image/california_wildfires/21_10_2017/921623155835224064_1.jpg
Processing: data_image/hurricane_harvey/10_9_2017/906792024094986240_0.jpg
Processing: data_image/hurricane_irma/19_9_2017/910118839081472000_0.jpg
Processing: data_image/iraq_iran_earthquake/18_11_2017/931898198569996288_0.jpg
Processing: data_image/hurricane_harvey/5_9_2017/905069428341174272_0.jpg
Processing: data_image/hurricane_irma/18_9_2017/909796661756653569_0.jpgProcessing: data

ValueError: Exception encountered when calling layer 'embeddings' (type TFViTEmbeddings).

too many values to unpack (expected 4)

Call arguments received by layer 'embeddings' (type TFViTEmbeddings):
  • pixel_values=tf.Tensor(shape=(36, 1, 3, 224, 224), dtype=float32)
  • interpolate_pos_encoding=None
  • training=False

In [2]:
def extract_dof_features(df, split_name, max_workers=4):
    output_file = f"{split_name}_dof_features.npy"
    if os.path.exists(output_file):
        print(f"✅ Loading cached {split_name} DOF features from {output_file}")
        return np.load(output_file)
    
    features = []
    failed_images = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_single_image, row): row["image"] for _, row in df.iterrows()}
        
        for future in as_completed(futures):
            try:
                result = future.result()
                if result is not None:
                    features.append(result)
                else:
                    failed_images.append(futures[future])
            except Exception as e:
                print(f"❌ Error on {futures[future]}: {e}")
                failed_images.append(futures[future])
    
    if len(features) == 0:
        raise ValueError("No features extracted. Exiting without saving.")

    features = np.array(features)
    np.save(output_file, features)
    print(f"\n✅ Saved {split_name} DOF features: {features.shape} → {output_file}")

    if failed_images:
        print(f"\n⚠️ {len(failed_images)} images failed. Saving failed list to {split_name}_failed.txt")
        with open(f"{split_name}_failed.txt", "w") as f:
            for fname in failed_images:
                f.write(fname + "\n")

    return features


In [ ]:
train_features = extract_dof_features(df_train, "train")
val_features = extract_dof_features(df_val, "val")
test_features = extract_dof_features(df_test, "test")


Processing: data_image/hurricane_harvey/8_9_2017/905960092822003712_0.jpg
Processing: data_image/california_wildfires/11_10_2017/918008272363368448_0.jpg
Processing: data_image/hurricane_irma/17_9_2017/909396901254090752_0.jpg
Processing: data_image/hurricane_maria/24_9_2017/912097936200355841_0.jpg


In [ ]:
# Visualization
print("\n\U0001F5BC Generating DGF visualizations...")
class_names = ["Severe", "Mild", "Little or None"]
class_images = {}
for label in [0, 1, 2]:
    rows = df_test[df_test["damage_label"] == label]
    if not rows.empty:
        class_images[label] = rows.iloc[0]["image"]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, label in enumerate([0, 1, 2]):
    img, img_tensor = load_image(class_images[label])
    _, dgf_map = compute_dof(img, img_tensor)
    axes[i].imshow(dgf_map, cmap='hot')
    axes[i].set_title(f"DGF - {class_names[i]}")
    axes[i].axis('off')
plt.tight_layout()
plt.savefig("dof_feature_visualizations_swin.png")
print("\u2705 Visualization saved as dgf_feature_visualizations_swin.png")


In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from datetime import datetime
from transformers import TFViTForImageClassification, ViTImageProcessor
from concurrent.futures import ThreadPoolExecutor, as_completed

print(f"\U0001F680 Starting DOF feature extraction at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Paths
image_dir = "/home/shahid/000.Damage Severity/aman/"
model_name = "microsoft/swin-tiny-patch4-window7-224"
model_save_path = "./swin_transformer_dof"

# Load TSVs
df_train = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_train.tsv"))
df_val = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_dev.tsv"))
df_test = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_test.tsv"))

label_mapping = {"severe_damage": 0, "mild_damage": 1, "little_or_no_damage": 2}
for df in [df_train, df_val, df_test]:
    df["damage_label"] = df["label"].map(label_mapping)

# Load model and processor
processor = ViTImageProcessor.from_pretrained(model_name)
model = TFViTForImageClassification.from_pretrained(model_name, num_labels=3, ignore_mismatched_sizes=True)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

if not os.path.exists(model_save_path):
    model.save(model_save_path)

# Load image and prepare input
def load_image(image_path):
    full_path = os.path.join(image_dir, image_path)
    if not os.path.exists(full_path):
        return None, None
    img = cv2.imread(full_path)
    if img is None:
        return None, None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (224, 224))
    inputs = processor(images=img_resized, return_tensors="tf")
    return img_resized, inputs["pixel_values"]

# Compute DOF features
def compute_dof(image, img_tensor):
    patch_size = 64
    stride = 32
    sensitivity_map = np.zeros((224, 224))
    
    base_pred = model(img_tensor).logits[0, 0].numpy()
    
    img_array = img_tensor.numpy()[0]  # shape: (3, 224, 224)

    patches = []
    patch_coords = []

    for i in range(0, 224 - patch_size + 1, stride):
        for j in range(0, 224 - patch_size + 1, stride):
            occluded = img_array.copy()
            occluded[:, i:i+patch_size, j:j+patch_size] = 0
            # Transpose to HWC for processor
            occluded_img = np.transpose(occluded, (1, 2, 0)).astype(np.uint8)
            patches.append(occluded_img)
            patch_coords.append((i, j))

    # Use processor to handle normalization, etc.
    inputs = processor(images=patches, return_tensors="tf")
    pixel_values = inputs["pixel_values"]  # shape: (N, 224, 224, 3)

    occluded_preds = model(pixel_values).logits[:, 0].numpy()

    for idx, (i, j) in enumerate(patch_coords):
        sensitivity_map[i:i+patch_size, j:j+patch_size] = np.abs(base_pred - occluded_preds[idx])

    features = [np.mean(sensitivity_map), np.max(sensitivity_map), np.std(sensitivity_map)]
    return np.nan_to_num(features), sensitivity_map


# Process single image
def process_single_image(row):
    try:
        print(f"Processing: {row['image']}")
        img, img_tensor = load_image(row['image'])
        if img is None or img_tensor is None:
            print(f"    ⚠️ Skipped (image not found or unreadable)")
            return None
        dof_feat, _ = compute_dof(img, img_tensor)
        print(f"    ✅ Feature extracted | Mean: {dof_feat[0]:.4f}")
        return dof_feat
    except Exception as e:
        print(f"    ❌ Error processing {row['image']}: {e}")
        return None

# Extract and save DOF features
def extract_dof_features(df, split_name, max_workers=4):
    output_file = f"{split_name}_dof_features.npy"
    if os.path.exists(output_file):
        print(f"✅ Loading cached {split_name} DOF features from {output_file}")
        return np.load(output_file)
    
    features = []
    failed_images = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_single_image, row): row["image"] for _, row in df.iterrows()}
        
        for future in as_completed(futures):
            try:
                result = future.result()
                if result is not None:
                    features.append(result)
                else:
                    failed_images.append(futures[future])
            except Exception as e:
                print(f"❌ Error on {futures[future]}: {e}")
                failed_images.append(futures[future])
    
    if len(features) == 0:
        raise ValueError(f"No features extracted for {split_name}. Exiting without saving.")

    features = np.array(features)
    np.save(output_file, features)
    print(f"\n✅ Saved {split_name} DOF features: {features.shape} → {output_file}")

    if failed_images:
        print(f"\n⚠️ {len(failed_images)} {split_name} images failed. Saved failed list to {split_name}_failed.txt")
        with open(f"{split_name}_failed.txt", "w") as f:
            for fname in failed_images:
                f.write(fname + "\n")

    return features

# -----------------------------------
# Extract features in this order:
# 1. Validation
# 2. Test
# 3. Training
# -----------------------------------

val_features = extract_dof_features(df_val, "val")
test_features = extract_dof_features(df_test, "test")
train_features = extract_dof_features(df_train, "train")

print("\n✅ All DOF features extracted and saved successfully!")


/home/shahid/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2025-07-03 11:39:20.138139: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 11:39:20.202613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 11:39:20.255350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 11:39:20.263471: E external/local_xla/xla/stream_exe

🚀 Starting DOF feature extraction at 2025-07-03 11:39:37


You are using a model of type swin to instantiate a model of type vit. This is not supported for all configurations of models and can yield errors.
2025-07-03 11:39:39.131226: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['swin.encoder.layers.2.blocks.3.intermediate.dense.bias', 'swin.encoder.layers.3.blocks.1.attention.self.key.weight', 'swin.encoder.layers.3.blocks.1.attention.self.query.weight', 'swin.encoder.layers.1.blocks.1.layernorm_before.bias', 'swin.encoder.layers.2.blocks.0.attention.self.relative_position_index', 'swin.encoder.layers.3.blocks.0.laye

Processing: data_image/california_wildfires/16_10_2017/919946322177802241_0.jpg
Processing: data_image/hurricane_harvey/10_9_2017/906842826457133056_0.jpg
Processing: data_image/hurricane_maria/6_10_2017/916148763105943552_0.jpg
Processing: data_image/hurricane_irma/7_9_2017/905631198843691009_0.jpg
    ✅ Feature extracted | Mean: 0.0235
Processing: data_image/hurricane_harvey/8_9_2017/906195501338558465_0.jpg
    ✅ Feature extracted | Mean: 0.0852
Processing: data_image/srilanka_floods/31_5_2017/870015964795908097_0.jpg
    ✅ Feature extracted | Mean: 0.5503
Processing: data_image/hurricane_irma/18_9_2017/909688022869868545_0.jpg
    ✅ Feature extracted | Mean: 0.0377
Processing: data_image/hurricane_irma/7_9_2017/905636873921495040_0.jpg
    ✅ Feature extracted | Mean: 0.1258
Processing: data_image/hurricane_harvey/6_9_2017/905427798466207746_0.jpg
    ✅ Feature extracted | Mean: 0.2535
Processing: data_image/hurricane_maria/1_10_2017/914329492830789634_2.jpg
    ✅ Feature extracted 

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from datetime import datetime
from transformers import TFViTForImageClassification, ViTImageProcessor
from concurrent.futures import ThreadPoolExecutor, as_completed

print(f"\U0001F680 Starting DOF feature extraction at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Paths
image_dir = "/home/shahid/000.Damage Severity/aman/"
model_name = "microsoft/swin-tiny-patch4-window7-224"
model_save_path = "./swin_transformer_dof"

# Load TSVs
df_train = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_train.tsv"))
df_val = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_dev.tsv"))
df_test = pd.read_table(os.path.join(image_dir, "embeddings/task_damage_text_img_test.tsv"))

label_mapping = {"severe_damage": 0, "mild_damage": 1, "little_or_no_damage": 2}
for df in [df_train, df_val, df_test]:
    df["damage_label"] = df["label"].map(label_mapping)

# Load model and processor
processor = ViTImageProcessor.from_pretrained(model_name)
model = TFViTForImageClassification.from_pretrained(model_name, num_labels=3, ignore_mismatched_sizes=True)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

if not os.path.exists(model_save_path):
    model.save(model_save_path)

# Load and preprocess image
def load_image(image_path):
    full_path = os.path.join(image_dir, image_path)
    if not os.path.exists(full_path):
        return None, None
    img = cv2.imread(full_path)
    if img is None:
        return None, None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (224, 224))
    inputs = processor(images=img_resized, return_tensors="tf")
    return img_resized, inputs["pixel_values"]

# Compute DOF with batch inference
def compute_dof(image, img_tensor, patch_size=96, stride=96, batch_size=16):
    sensitivity_map = np.zeros((224, 224))
    base_pred = model(img_tensor).logits[0, 0].numpy()
    
    img_array = img_tensor.numpy()[0]  # (3, 224, 224)
    patches = []
    patch_coords = []

    for i in range(0, 224 - patch_size + 1, stride):
        for j in range(0, 224 - patch_size + 1, stride):
            occluded = img_array.copy()
            occluded[:, i:i+patch_size, j:j+patch_size] = 0
            occluded_img = np.transpose(occluded, (1, 2, 0)).astype(np.uint8)
            patches.append(occluded_img)
            patch_coords.append((i, j))

    # Process occluded images in batches
    occluded_preds = []
    for k in range(0, len(patches), batch_size):
        batch_imgs = patches[k:k+batch_size]
        batch_inputs = processor(images=batch_imgs, return_tensors="tf")
        batch_pixel_values = batch_inputs["pixel_values"]
        batch_logits = model(batch_pixel_values).logits[:, 0].numpy()
        occluded_preds.extend(batch_logits)

    # Sensitivity map calculation
    for idx, (i, j) in enumerate(patch_coords):
        sensitivity_map[i:i+patch_size, j:j+patch_size] = np.abs(base_pred - occluded_preds[idx])

    features = [np.mean(sensitivity_map), np.max(sensitivity_map), np.std(sensitivity_map)]
    return np.nan_to_num(features), sensitivity_map

# Process one image
def process_single_image(row):
    try:
        print(f"Processing: {row['image']}")
        img, img_tensor = load_image(row['image'])
        if img is None or img_tensor is None:
            print(f"    ⚠️ Skipped (image not found or unreadable)")
            return None
        dof_feat, _ = compute_dof(img, img_tensor)
        print(f"    ✅ Feature extracted | Mean: {dof_feat[0]:.4f}")
        return dof_feat
    except Exception as e:
        print(f"    ❌ Error processing {row['image']}: {e}")
        return None

# Extract and save DOF features
def extract_dof_features(df, split_name, max_workers=4):
    output_file = f"{split_name}_dof_features.npy"
    if os.path.exists(output_file):
        print(f"✅ Loading cached {split_name} DOF features from {output_file}")
        return np.load(output_file)
    
    features = []
    failed_images = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_single_image, row): row["image"] for _, row in df.iterrows()}
        
        for future in as_completed(futures):
            try:
                result = future.result()
                if result is not None:
                    features.append(result)
                else:
                    failed_images.append(futures[future])
            except Exception as e:
                print(f"❌ Error on {futures[future]}: {e}")
                failed_images.append(futures[future])

            # Save checkpoint every 50 samples
            if len(features) > 0 and len(features) % 50 == 0:
                np.save(f"{split_name}_checkpoint.npy", np.array(features))

    if len(features) == 0:
        raise ValueError(f"No features extracted for {split_name}. Exiting without saving.")

    features = np.array(features)
    np.save(output_file, features)
    print(f"\n✅ Saved {split_name} DOF features: {features.shape} → {output_file}")

    if failed_images:
        print(f"\n⚠️ {len(failed_images)} {split_name} images failed. Saved list to {split_name}_failed.txt")
        with open(f"{split_name}_failed.txt", "w") as f:
            for fname in failed_images:
                f.write(fname + "\n")

    return features

# ---------------------
# Run in this order
# ---------------------
val_features = extract_dof_features(df_val, "val")
test_features = extract_dof_features(df_test, "test")
train_features = extract_dof_features(df_train, "train")

print("\n✅ All DOF features extracted and saved successfully!")


/home/shahid/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2025-07-03 11:50:52.384483: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 11:50:52.447237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 11:50:52.487825: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 11:50:52.497026: E external/local_xla/xla/stream_exe

🚀 Starting DOF feature extraction at 2025-07-03 11:51:13


You are using a model of type swin to instantiate a model of type vit. This is not supported for all configurations of models and can yield errors.
2025-07-03 11:51:14.549755: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['swin.encoder.layers.2.blocks.3.attention.self.relative_position_bias_table', 'swin.encoder.layers.2.blocks.2.attention.self.key.weight', 'swin.encoder.layers.3.blocks.0.layernorm_after.weight', 'swin.encoder.layers.2.blocks.5.attention.self.value.weight', 'swin.encoder.layers.2.blocks.3.attention.self.query.bias', 'swin.encoder.layers.0.block

Processing: data_image/california_wildfires/16_10_2017/919946322177802241_0.jpg
Processing: data_image/hurricane_harvey/10_9_2017/906842826457133056_0.jpg
Processing: data_image/hurricane_maria/6_10_2017/916148763105943552_0.jpg
Processing: data_image/hurricane_irma/7_9_2017/905631198843691009_0.jpg
    ✅ Feature extracted | Mean: 0.9011
Processing: data_image/hurricane_harvey/8_9_2017/906195501338558465_0.jpg
    ✅ Feature extracted | Mean: 0.9988
Processing: data_image/srilanka_floods/31_5_2017/870015964795908097_0.jpg
    ✅ Feature extracted | Mean: 0.7646
Processing: data_image/hurricane_irma/18_9_2017/909688022869868545_0.jpg
    ✅ Feature extracted | Mean: 0.2328
Processing: data_image/hurricane_irma/7_9_2017/905636873921495040_0.jpg
    ✅ Feature extracted | Mean: 0.7591
Processing: data_image/hurricane_harvey/6_9_2017/905427798466207746_0.jpg
    ✅ Feature extracted | Mean: 0.0982
Processing: data_image/hurricane_maria/1_10_2017/914329492830789634_2.jpg
    ✅ Feature extracted 